Test performance of RoBerta on tweet_eval Emotion Dataset

Library packages

In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim 
import numpy as np 
import matplotlib.pyplot as plt
import nltk
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, roc_curve, auc
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
import os
from sklearn.metrics import balanced_accuracy_score
!pip install autocorrect
import pickle
from autocorrect import Speller
import re
import string
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer 
!pip install emoji
import emoji


     |████████████████████████████████| 622 kB 5.0 MB/s 
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622382 sha256=4aa3a2823b61158f8b48a69522e53b6d26e5ee9dd7ed134b51b382433baff385
  Stored in directory: /root/.cache/pip/wheels/54/d4/37/8244101ad50b0f7d9bffd93ce58ed7991ee1753b290923934b
Successfully built autocorrect
     |████████████████████████████████| 175 kB 5.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=d1143302bec2e18ea0251264d6abd418a5695b52c2c2ee728e1a6f691fb1ff23
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [4]:
## the HuggingFace libraries are not already installed in Colab
!pip install transformers==4.17 # this will also install tokenizers 
!pip install datasets

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 77 kB 9.1 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
     |████████████████████████████████| 596 kB 68.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 66.1 MB/s 
     |████████████████████████████████| 212 kB 75.1 MB/s 
     |████████████████████████████████| 136 kB 78.3 MB/s 
     |████████████████████████████████| 127 kB 77.2 MB/s 
     |████████████████████████████████| 144 kB 63.5 MB/s 
     |████████████████████████████████| 271 kB 56.0 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [5]:
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, RobertaForSequenceClassification,RobertaModel, Trainer, TrainingArguments
from transformers import AdamW, get_linear_schedule_with_warmup
import datasets
print(transformers.__version__)

4.17.0


In [26]:
#Installing emot library
# !pip install emot #Importing libraries
# !pip install emot.emo_unicode
import re
from emot.emo_unicode import UNICODE_EMOJI

Load data

In [6]:
data = datasets.load_dataset("tweet_eval", "emotion")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
### number of labels
num_labels = len(np.unique(data['train']['label']))
print (num_labels)

4


In [8]:
# % splits of labels in train/validation/test
from collections import OrderedDict, Counter
class OrderedCounter(Counter, dict):
    'Counter that remembers the order elements are first encountered'
    def __repr__(self):
        return '%s(%r)' % (self.__class__.__name__, dict(self))

    def __reduce__(self):
        return self.__class__, (dict(self),)
data_len_train = len(data['train']['label'])
# print (data_len_train)
data_len_val = len(data['validation']['label'])
data_len_test = len(data['test']['label'])
# print (OrderedCounter(data['train']['label'])[19])

for i in range(num_labels):
  print ('Train labels : {}, percentage: {:.2f}'.format(i,(OrderedCounter(data['train']['label'])[i]/data_len_train)*100))
  print ('Val labels : {}, percentage: {:.2f}'.format(i,(OrderedCounter(data['validation']['label'])[i]/data_len_val)*100))
  print ('Test labels : {}, percentage: {:.2f}'.format(i,(OrderedCounter(data['test']['label'])[i]/data_len_test)*100))


Train labels : 0, percentage: 42.98
Val labels : 0, percentage: 42.78
Test labels : 0, percentage: 39.27
Train labels : 1, percentage: 21.74
Val labels : 1, percentage: 25.94
Test labels : 1, percentage: 25.19
Train labels : 2, percentage: 9.03
Val labels : 2, percentage: 7.49
Test labels : 2, percentage: 8.66
Train labels : 3, percentage: 26.25
Val labels : 3, percentage: 23.80
Test labels : 3, percentage: 26.88


In [97]:
class CleanedEmotionsDataset():

    def __init__(self, tokenizer, df,max_length):
        super(CleanedEmotionsDataset, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)
        

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq) # convert to id
        input_ids += padding   # add padding
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len) # for identifying next sentence prediction
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids 

    # strip  @ with words and numbers
    def remove_pattern(self,input_txt):
        r = re.findall("@[\w]*", input_txt)
        for i in r:
            input_txt = re.sub(i, '', input_txt)
        input_txt = re.sub(r"\d+", "", input_txt)
        return input_txt  

    ## strip all @ and # with words following
    def strip_links(self,text):
        link_regex = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
        links = re.findall(link_regex, text)
        for link in links:
            text = text.replace(link[0], ', ')    
        return text
        
    def strip_all_entities(self,text):
        entity_prefixes = ['@','#']
        for separator in  string.punctuation:
            if separator not in entity_prefixes :
                text = text.replace(separator,' ')
        words = []
        for word in text.split():
            word = word.strip()
            if word:
                if word[0] not in entity_prefixes:
                    words.append(word)
        return ' '.join(words)

        # removing stopwords
    def removing_stopwords(self,text):
        text = repr(text)
        No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
        # Convert list of tokens_without_stopwords to String type.
        words_string = ' '.join(No_StopWords)    
        return words_string

    ## lemmatization
    def lemmatization(self,text):
        # Converting words to their root forms
        lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
        return ' '.join(lemma)
    
    ## convert emoji to text
    def convert_emojis(self,text):
        for emot in UNICODE_EMOJI:
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
            text = text.replace('_', " ")
        return text

    def get_input(self,df):
        # df_tweet = self.clean_dataset(df['text'])
        ## clean off tags

        for i in range(len(df['text'])):
            df['text'][i] = df['text'][i].lower()
            df['text'][i] = self.remove_pattern(df['text'][i])
            df['text'][i] = self.convert_emojis(df['text'][i])
            # df['text'][i] = self.lemmatization(df['text'][i])
            # df['text'][i] = self.removing_stopwords(df['text'][i])
            # df['text'][i] = self.spell(self.lemmatization(df['text'][i]))
            
        tweet = df['text'].values
        # print (tweet)
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]

Model Class

In [36]:
class FinetuneModel(nn.Module):
    """Pretrained Model + classifier
    """
    def __init__(self, pretrained_model, d_in, d_h, d_out,dropout,freeze_pretrained=False,model_dir = None):
        super(FinetuneModel, self).__init__()

        # Instantiate BERT model
        
        self.pretrained_model = pretrained_model
        if model_dir != None:
            print ('True')
            self.pretrained_model.load_state_dict(model_dir,strict = False)

        # Instantiate a 2-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_in, d_h),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_h, d_out)
        )

        # Freeze the encoder model
        if freeze_pretrained:
            for param in self.pretrained_model.parameters():
                param.requires_grad = False
            for param in self.pretrained_model.pooler.parameters():
                param.requires_grad = True
        
    def forward(self, input_ids, attention_mask):
        # Feed input to pretrained model
        pre_outputs = self.pretrained_model(input_ids = input_ids,attention_mask = attention_mask)
        # print (pre_outputs[0].shape,pre_outputs[1].shape)
        # print (pre_outputs.keys())
        hidden_state = pre_outputs[0]
        hidden_state = hidden_state[:,0,:]
        logits = self.classifier(hidden_state)
        # print(outputs.keys()) # ["last_hidden_state", "pooler_output", "hidden_states", "attention"] last hidden state 才是最后的hidden
        # print (outputs[0].shape)
        # Extract the last hidden state
        # last_hidden = outputs["last_hidden_state"] # shape: (batch_size, sequence_length, d_in)
        
        # # Use the `[CLS]` token of the last hidden state for classification task
        # last_hidden_state_cls = last_hidden[:, 0, :] # shape: (batch_size, d_in)

        # Feed input to classifier to compute logits
        # logits = self.classifier(outputs[0]) # shape: (batch_size, 1)

        return logits

Function to init model

In [57]:
def model_init(model_name,lr,weight_decay,batch_size,warmup,data):
  pretrained_model_emotions = RobertaModel.from_pretrained(model_name,config=config)
  model = FinetuneModel(pretrained_model_emotions,d_in,d_h,d_out,dropout,freeze_pretrained)
  model.to(device)
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
          {
                  'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                  'weight_decay':weight_decay
          },
          {
                  'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                  'weight_decay':0.0
          }
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr, betas = (0.9,0.98),eps = 1e-6)
  total_steps = int(n_epochs * (len(data["train"]["text"]) / batch_size))
  warmup_steps = int(warmup * total_steps)
  scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps = warmup_steps, 
      num_training_steps = total_steps)
  
  return model,optimizer,scheduler


Training and Eval function

In [69]:
def set_seed(seed_value = 0):
    """Set seed for reproducibility.
    """
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(train_loader, model, optimizer, scheduler,loss_fct):
    model.train()
    train_losses = []
    correct_preds = 0
    pred_class = []
    actual_class = []
    tqdm_loader = tqdm(train_loader)
    for batch_index, (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in enumerate (tqdm_loader):
        model.zero_grad()
  

        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        segments = segments.to(device)
        labels = labels.to(device)

        logits = model(input_ids, attention_mask) # forward pass (batch_size, 1)
        loss = loss_fct(logits,labels)
        probs = nn.functional.softmax(logits,dim = -1)
        _,out_classes = torch.max(logits,dim = 1)
        ## for weighted accuracy
        pred_class.extend(out_classes.cpu().numpy())
        actual_class.extend(labels.cpu().numpy())

        correct = (out_classes == labels).sum()
        correct_preds += correct.item()
        train_losses.append(loss.item())

        loss.backward() # backward pass
        ## clip grad
        nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step() # weights update
        scheduler.step()
        optimizer.zero_grad()
        
    train_loss = np.mean(train_losses)
    epoch_accuracy = correct_preds /len(train_loader.dataset)
    # epoch_accuracy = balanced_accuracy_score(actual_class,pred_class)

    return train_loss,epoch_accuracy

In [59]:
from sklearn.metrics import roc_auc_score, f1_score
def eval_multiclass(loader, model,loss_fct):
    model.eval()
    val_losses = []
    val_probs = []
    acc_correct = 0
    iter = 0
    log_interval = 100
    sigmoid = nn.Sigmoid()
    log_interval = 50
    count = 0
    val_labels = []
    val_targets = []
    pred_class = []
    actual_class = []
  
    # threshold = torch.tensor([0.5]).to(device)
    # loader = tqdm(loader)
    for (batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels) in loader:
        input_ids = batch_seqs
        attention_mask = batch_seq_masks
        segments = batch_seq_segments
        labels = batch_labels

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        segments = segments.to(device)
        # print (attention_mask.shape,labels.shape)
        # print (segments.shape)
        
        with torch.no_grad():
            valid_logits = model(input_ids, attention_mask) # raw logits
            # print (f'logits = {valid_logits}')
            # print(f'labels = {labels}')
            valid_loss = loss_fct(valid_logits,labels) # loss
            valid_probs = nn.functional.softmax(valid_logits,dim = -1)
            _,out_classes = torch.max(valid_logits,dim = 1)
            # if count % log_interval == 0:
            #   print (f'labels = {labels}')
            #   print (f'predicted labels = {out_classes}')
            f1_target = torch.argmax(valid_probs,dim= 1)
            # print (f1_target)
            correct = (out_classes == labels).sum()
            acc_correct += correct.item()
            # weighted accuracy

            pred_class.extend(out_classes.cpu().numpy())
            actual_class.extend(labels.cpu().numpy())


            # print (valid_probs.shape)
            val_probs.extend(valid_probs[:,:].cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_losses.append(valid_loss.item())
            val_targets.extend(f1_target.cpu().numpy())
            count +=1
            # print (count)
    val_loss = np.mean(val_losses)
    val_probs = np.array(val_probs)
    val_labels = np.array(val_labels)
    val_targets = np.array(val_targets)
    valid_accuracy = acc_correct/(len(loader.dataset))
    # valid_accuracy = balanced_accuracy_score(actual_class,pred_class)

    
    val_auc = 100* roc_auc_score(val_labels, val_probs, multi_class='ovr')
    val_f1_score = f1_score(val_labels,val_targets,average = 'weighted')

    return val_loss, val_auc, val_targets, val_labels,valid_accuracy,val_f1_score

Hyperparameters 

In [65]:
# Hyper-parameters
model_name = "roberta-base" # we can change this, and play with any model from HuggingFace! 
# BERT-base: "bert-base-cased" #Sensitive to lower and upper case
# BERT-large: "bert-large-cased"
# RoBERTa-base: "roberta-base"
# RoBERTa-large: "roberta-large"
# etc ... 
max_length = 50
batch_size = 32
d_in = 768 # bert-base / roberta-base
d_h = 512 # 512 
d_out = num_labels # 2 classes so 1 is enough
dropout = 0.1
freeze_pretrained = False

# Training and Optimizer HP
# Hyper-parameters
n_epochs = 10
lr = 3e-05
warmup = 0.06
max_gradient_norm = 10
weight_decay = 0.1



Tokenizer

In [38]:
tokenizer = AutoTokenizer.from_pretrained( # robertatokenizer fast
    model_name,
    cache_dir = "."
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Initialise EmotionDataset and dataloader

In [98]:
stoplist = stopwords.words('english') 
stoplist = set(stoplist)
## lemmatization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
## speller
speller = Speller(lang = 'en')
# Convert to pandas df
data_train = datasets.Dataset.to_pandas(data["train"])
data_val = datasets.Dataset.to_pandas(data["validation"])
data_test = datasets.Dataset.to_pandas(data["test"])

train_dataset = CleanedEmotionsDataset(tokenizer, data_train,max_length)
val_dataset = CleanedEmotionsDataset(tokenizer, data_val,max_length)
test_dataset = CleanedEmotionsDataset(tokenizer, data_test,max_length)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [99]:
# data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [100]:
## visualise text
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
        text = text.replace('_', " ")
    return text
def remove_pattern(input_txt):
    r = re.findall("@[\w]*", input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    input_txt = re.sub(r"\d+", "", input_txt)
    return input_txt

def removing_stopwords(text):
    text = repr(text)
    No_StopWords = [word for word in word_tokenize(text) if word.lower() not in stoplist ]
    # Convert list of tokens_without_stopwords to String type.
    words_string = ' '.join(No_StopWords)    
    return words_string

def lemmatization(text):
    # Converting words to their root forms
    lemma = [lemmatizer.lemmatize(w,'v') for w in w_tokenizer.tokenize(text)]
    return ' '.join(lemma)

text = data['train']['text'][32]
print (f'Original text: {text}')
print (tokenizer.tokenize(text))

clean_text = convert_emojis(remove_pattern(text.lower()))
print (f'Cleaned text: {clean_text}')
print (tokenizer.tokenize(clean_text))



Original text: @user 'shit' doesn't even begin to describe these fiery little demons straight from hell 🌝🌚 ;)
['@', 'user', "Ġ'", 'shit', "'", 'Ġdoesn', "'t", 'Ġeven', 'Ġbegin', 'Ġto', 'Ġdescribe', 'Ġthese', 'Ġfiery', 'Ġlittle', 'Ġdemons', 'Ġstraight', 'Ġfrom', 'Ġhell', 'ĠðŁ', 'Į', 'Ŀ', 'ðŁ', 'Į', 'ļ', 'Ġ;)']
Cleaned text:  'shit' doesn't even begin to describe these fiery little demons straight from hell full moon facenew moon face ;)
["Ġ'", 'shit', "'", 'Ġdoesn', "'t", 'Ġeven', 'Ġbegin', 'Ġto', 'Ġdescribe', 'Ġthese', 'Ġfiery', 'Ġlittle', 'Ġdemons', 'Ġstraight', 'Ġfrom', 'Ġhell', 'Ġfull', 'Ġmoon', 'Ġfac', 'en', 'ew', 'Ġmoon', 'Ġface', 'Ġ;)']


Config for Roberta model

In [66]:
### config
config = AutoConfig.from_pretrained(
    model_name, 
    output_hidden_states = True,
    output_attention = False
) 
# remember AutoConfig
config

RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [101]:
### Evaluate on clean data
# set seed
loss_fct = nn.CrossEntropyLoss()
set_seed(seed_value = 0)
model,optimizer,scheduler = model_init(model_name,lr,weight_decay,batch_size,warmup,data)
val_aucs = []
val_accs = []
best_score = 0.
patience_counter = 0
val_loss, val_auc, _, _, val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
val_aucs.append(val_auc)
val_accs.append(val_acc)
print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}, f1_score : {:.2f}".format(val_loss, val_auc,val_acc*100,val_f1_score))
for epoch in range(1, n_epochs + 1):
    
    print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

    # Training round
    train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler,loss_fct)

    # Validation round
    val_loss, val_auc, _, _,val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
    val_aucs.append(val_auc)

    print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
  
    print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, f1_score : {:.2f} (best ACC: {:.4f} at epoch {})"
    .format(val_loss, val_auc,val_acc*100,val_f1_score,np.max(np.array(val_acc)),np.argmax(np.array(val_acc*100))))

    print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
    if val_auc == np.max(np.array(val_aucs)):
        print("!! New best val AUC !!")
    if val_acc == np.max(np.array(val_acc)):
        print("!! New best val ACC !!")

## eval on test dataset
_, test_auc, test_probs, test_labels,test_acc,test_f1_score = eval_multiclass(test_loader, model,loss_fct)
print ("test AUC : {:.2f}".format(test_auc))
print ("test f1 score : {:.2f}".format(test_f1_score))
print ("test ACC : {:.2f}".format(test_acc*100))
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Futu


Epoch 0 (before training) val loss: 1.4041, AUC: 46.81, ACC: 7.4866, f1_score : 0.01

New epoch, epoch 1 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 1.0954, accuracy 50.5373
Validation: loss: 0.6830, AUC: 90.76, ACC : 75.9358, f1_score : 0.74 (best ACC: 0.7594 at epoch 0)
lr : 2.8713389121338915e-05
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

...

Training: loss: 0.0524, accuracy 98.3727
Validation: loss: 0.8499, AUC: 94.41, ACC : 81.0160, f1_score : 0.81 (best ACC: 0.8102 at epoch 0)
lr : 6.3075313807531386e-06
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 0.0415, accuracy 98.4955
Validation: loss: 0.8866, AUC: 94.19, ACC : 79.1444, f1_score : 0.79 (best ACC: 0.7914 at epoch 0)
lr : 3.1066945606694563e-06
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 0.0263, accuracy 99.1096
Validation: loss: 0.9033, AUC: 94.28, ACC : 81.0160, f1_score : 0.81 (best ACC: 0.8102 at epoch 0)
lr : 0.0
!! New best val ACC !!
test AUC : 94.58
test f1 score : 0.83
test ACC : 82.90


In [102]:
n_check = len(data["test"]["text"])
total_examples = dict()
total_correct = dict()
for i in range(n_check):
    test_label = test_labels[i]
    prob = test_probs[i]
    total_examples[test_label] = total_examples.get(test_label,0) + 1
    if test_label == int(prob):
      total_correct[int(prob)] = total_correct.get(int(prob),0) + 1
# print (total_examples)
# print (total_correct)
for i in range(num_labels):
  print ('total class {} correct is : {:.2f}%'.format(i,(total_correct[i]/total_examples[i])*100))

## class 2 lowest score

total class 0 correct is : 85.30%
total class 1 correct is : 81.84%
total class 2 correct is : 65.85%
total class 3 correct is : 85.86%


Check how it classifies text

In [106]:
# Qualitative test samples
n_check = 10
label_name = dict({'0.0':'anger','1.0':'joy','2.0':'optimisim','3.0':'sadness'})
count = 5
for i in range(n_check):
    i+= i*(5+i)
    tweet = data["test"]["text"][i]
    test_label = test_labels[i]  
    prob = test_probs[i]
    print("\n" + "*"*20, "Tweet {}:".format(i+1))
    print("\n".join(sent_tokenize(tweet))) # print 1 sentence per line for reading easiness
    print("*"*10, "Label: {}".format(label_name[str(float(test_label))]))
    print ("*"*10, "Label: {}".format(test_label))
    print("*"*10, "Model prediction: {:.4f}".format(prob))


******************** Tweet 1:
#Deppression is real.
Partners w/ #depressed people truly dont understand the depth in which they affect us.
Add in #anxiety &amp;makes it worse
********** Label: sadness
********** Label: 3
********** Model prediction: 3.0000

******************** Tweet 8:
In need of a change!
#restless
********** Label: sadness
********** Label: 3
********** Model prediction: 3.0000

******************** Tweet 17:
Its #amazing watching various news outlets showing mixed crowds of ppl watching #Eclipse with NO #racial tension.. #MSM can't hide this!
********** Label: joy
********** Label: 1
********** Model prediction: 1.0000

******************** Tweet 28:
I cannot see a rational way of bearing a grudge to him for that.
I do not.
Maybe because I do not live by Netflix and I do not care...
********** Label: anger
********** Label: 0
********** Model prediction: 0.0000

******************** Tweet 41:
BUT, I have offended so many people with the idea that conflicts and val

Performance on original text

In [76]:
class EmotionsDataset_Original():

    def __init__(self, tokenizer, df,max_length):
        super(EmotionsDataset_Original, self).__init__()

        self.tokenizer = tokenizer
        self.max_seq_len = max_length
        self.input_ids, self.attention_mask,self.token_type_ids,self.label = self.get_input(df)
        

    def __len__(self):
        return len(self.label)
    
    def trunate_and_pad(self, tokens_seq):
        
        # Concat '[CLS]' at the beginning
        tokens_seq = ['[CLS]'] + tokens_seq     
        # Truncate sequences of which the lengths exceed the max_seq_len
        if len(tokens_seq) > self.max_seq_len:
            tokens_seq = tokens_seq[0 : self.max_seq_len]           
        # Generate padding
        padding = [0] * (self.max_seq_len - len(tokens_seq))       
        # Convert tokens_seq to token_ids
        input_ids = self.tokenizer.convert_tokens_to_ids(tokens_seq) # convert to id
        input_ids += padding   # add padding
        # Create attention_mask
        attention_mask = [1] * len(tokens_seq) + padding     
        # Create token_type_ids
        token_type_ids = [0] * (self.max_seq_len) # for identifying next sentence prediction
        
        assert len(input_ids) == self.max_seq_len
        assert len(attention_mask) == self.max_seq_len
        assert len(token_type_ids) == self.max_seq_len
        
        return input_ids, attention_mask, token_type_ids 

    def get_input(self,df):          
        tweet = df['text'].values
        label = df['label'].values
        tokens_seq = list(map(self.tokenizer.tokenize,tweet))
        result = list(map(self.trunate_and_pad, tokens_seq))
        input_ids = [i[0] for i in result]
        attention_mask = [i[1] for i in result]
        token_type_ids = [i[2] for i in result]

        return (
               torch.Tensor(input_ids).type(torch.long), 
               torch.Tensor(attention_mask).type(torch.long),
               torch.Tensor(token_type_ids).type(torch.long), 
               torch.Tensor(label).type(torch.long)
               )

    def __getitem__(self, item):
        return self.input_ids[item], self.attention_mask[item],self.token_type_ids[item],self.label[item]

In [77]:
data_train = datasets.Dataset.to_pandas(data["train"])
data_val = datasets.Dataset.to_pandas(data["validation"])
data_test = datasets.Dataset.to_pandas(data["test"])

train_dataset = EmotionsDataset_Original(tokenizer, data_train,max_length)
val_dataset = EmotionsDataset_Original(tokenizer, data_val,max_length)
test_dataset = EmotionsDataset_Original(tokenizer, data_test,max_length)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True) # shuffle the training set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

In [78]:
### Evaluate on original text
# set seed
loss_fct = nn.CrossEntropyLoss()
set_seed(seed_value = 0)
model,optimizer,scheduler = model_init(model_name,lr,weight_decay,batch_size,warmup,data)
val_aucs = []
val_accs = []
best_score = 0.
patience_counter = 0
val_loss, val_auc, _, _, val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
val_aucs.append(val_auc)
val_accs.append(val_acc)
print("\nEpoch 0 (before training) val loss: {:.4f}, AUC: {:.2f}, ACC: {:.4f}, f1_score : {:.2f}".format(val_loss, val_auc,val_acc*100,val_f1_score))
for epoch in range(1, n_epochs + 1):
    
    print("\nNew epoch, epoch {} / {}".format(epoch, n_epochs))

    # Training round
    train_loss,train_accuracy = train(train_loader, model, optimizer, scheduler,loss_fct)

    # Validation round
    val_loss, val_auc, _, _,val_acc,val_f1_score= eval_multiclass(val_loader, model,loss_fct)
    val_aucs.append(val_auc)

    print("Training: loss: {:.4f}, accuracy {:.4f}".format(train_loss,train_accuracy*100))
  
    print ("Validation: loss: {:.4f}, AUC: {:.2f}, ACC : {:.4f}, f1_score : {:.2f} (best ACC: {:.4f} at epoch {})"
    .format(val_loss, val_auc,val_acc*100,val_f1_score,np.max(np.array(val_acc)),np.argmax(np.array(val_acc*100))))

    print ('lr : {}'.format(scheduler.optimizer.param_groups[0]['lr']))
    if val_auc == np.max(np.array(val_aucs)):
        print("!! New best val AUC !!")
    if val_acc == np.max(np.array(val_acc)):
        print("!! New best val ACC !!")

## eval on test dataset
_, test_auc, test_probs, test_labels,test_acc,test_f1_score = eval_multiclass(test_loader, model,loss_fct)
print ("test AUC : {:.2f}".format(test_auc))
print ("test f1 score : {:.2f}".format(test_f1_score))
print ("test ACC : {:.2f}".format(test_acc*100))
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Futu


Epoch 0 (before training) val loss: 1.4029, AUC: 49.07, ACC: 7.4866, f1_score : 0.01

New epoch, epoch 1 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 1.0846, accuracy 51.7347
Validation: loss: 0.6820, AUC: 90.99, ACC : 75.6684, f1_score : 0.75 (best ACC: 0.7567 at epoch 0)
lr : 2.8713389121338915e-05
!! New best val AUC !!
!! New best val ACC !!

New epoch, epoch 2 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

...

Training: loss: 0.0563, accuracy 98.3420
Validation: loss: 1.0416, AUC: 92.97, ACC : 77.5401, f1_score : 0.78 (best ACC: 0.7754 at epoch 0)
lr : 6.3075313807531386e-06
!! New best val ACC !!

New epoch, epoch 9 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 0.0374, accuracy 98.8947
Validation: loss: 1.0087, AUC: 92.88, ACC : 79.4118, f1_score : 0.80 (best ACC: 0.7941 at epoch 0)
lr : 3.1066945606694563e-06
!! New best val ACC !!

New epoch, epoch 10 / 10


  0%|          | 0/102 [00:00<?, ?it/s]

Training: loss: 0.0321, accuracy 99.1710
Validation: loss: 1.0452, AUC: 92.88, ACC : 79.1444, f1_score : 0.79 (best ACC: 0.7914 at epoch 0)
lr : 0.0
!! New best val ACC !!
test AUC : 94.50
test f1 score : 0.82
test ACC : 81.63


In [79]:
# Qualitative test samples
n_check = 10
label_name = dict({'0.0':'anger','1.0':'joy','2.0':'optimisim','3.0':'sadness'})
for i in range(n_check):
    tweet = data["test"]["text"][i]
    test_label = test_labels[i]  
    prob = test_probs[i]
    print("\n" + "*"*20, "Tweet {}:".format(i+1))
    print("\n".join(sent_tokenize(tweet))) # print 1 sentence per line for reading easiness
    print("*"*10, "Label: {}".format(label_name[str(float(test_label))]))
    print ("*"*10, "Label: {}".format(test_label))
    print("*"*10, "Model prediction: {:.4f}".format(prob))


******************** Tweet 1:
#Deppression is real.
Partners w/ #depressed people truly dont understand the depth in which they affect us.
Add in #anxiety &amp;makes it worse
********** Label: sadness
********** Label: 3
********** Model prediction: 3.0000

******************** Tweet 2:
@user Interesting choice of words... Are you confirming that governments fund #terrorism?
Bit of an open door, but still...
********** Label: anger
********** Label: 0
********** Model prediction: 0.0000

******************** Tweet 3:
My visit to hospital for care triggered #trauma from accident 20+yrs ago and image of my dead brother in it.
Feeling symptoms of #depression
********** Label: sadness
********** Label: 3
********** Model prediction: 3.0000

******************** Tweet 4:
@user Welcome to #MPSVT!
We are delighted to have you!
#grateful #MPSVT #relationships
********** Label: joy
********** Label: 1
********** Model prediction: 1.0000

******************** Tweet 5:
What makes you feel #joyfu

Percentage accuracy in each class

In [80]:
n_check = len(data["test"]["text"])
total_examples = dict()
total_correct = dict()
for i in range(n_check):
    test_label = test_labels[i]
    prob = test_probs[i]
    total_examples[test_label] = total_examples.get(test_label,0) + 1
    if test_label == int(prob):
      total_correct[int(prob)] = total_correct.get(int(prob),0) + 1
# print (total_examples)
# print (total_correct)
for i in range(num_labels):
  print ('total class {} correct is : {:.2f}%'.format(i,(total_correct[i]/total_examples[i])*100))

total class 0 correct is : 84.05%
total class 1 correct is : 82.12%
total class 2 correct is : 70.73%
total class 3 correct is : 81.15%
